In [94]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import datetime
import time
import csv
import re

driver = webdriver.Chrome()
driver.set_window_size(1400, 1000)

driver.get('https://www.bd119.com/realty/realty_list.asp?RealtyType=L&sido=&gugun=&price1_search=&search_size=&sub_category=&x=10&y=13')

매물종류 / 소재지 / 면적 (연, 토지) / 가격 / 층수(방수) / 진입/인접도로 / 이삿날(주차) / 조회 / 제목 / 장점 / url

In [95]:
def crawl_page(driver, csv_writer):
    # 매물종류
    types = [type.text.replace("[", "").replace("]", "") for type in driver.find_elements(By.CSS_SELECTOR, "p[style='font-weight:bold;float:left;']")]

    # 소재지
    addresses = [address.text for address in driver.find_elements(By.CLASS_NAME, "realty_address")]
    urls = [address.find_element(By.TAG_NAME, "a").get_attribute('href') for address in driver.find_elements(By.CLASS_NAME, "realty_address")]
    address_parts = [address.split() for address in addresses]
    address1 = [part[0] if len(part) > 0 else '' for part in address_parts]
    address2 = [part[1] if len(part) > 1 else '' for part in address_parts]
    address3 = [' '.join(part[2:]) if len(part) > 2 else '' for part in address_parts]


    # 면적(연, 토지)
    sizes = [size.text for size in driver.find_elements(By.CSS_SELECTOR, "td[style='padding-left:5px;border-bottom:none;']")]

    # 가격
    price_elements = driver.find_elements(By.CSS_SELECTOR, "td[align='left'][style='border-bottom:none;']")
    prices = [price_elements[i].text for i in range(len(price_elements)) if i % 2 == 1]

    # 층수/방수
    rooms = [room.text for room in driver.find_elements(By.ID, "cell1")[1:]]

    # 진입/인접도로
    roads = [road.text for road in driver.find_elements(By.ID, "cell2")]

    # 이삿날/주차 (날짜 형식의 문자열 제거)
    parks = driver.find_elements(By.ID, "cell7")[1:]
    clean_parks = [re.sub(r'\d{4}-\d{2}-\d{2}', '', park.text).strip() for park in parks]
    clean_parks = [re.sub(r'\d{4}-\d{2}-', '', park).strip() for park in clean_parks]

    # 조회
    counts = [count.text.replace(" 회", "").strip() for count in driver.find_elements(By.ID, "cell_hit")[1:]]

    # 제목 및 URL
    title_elements = driver.find_elements(By.ID, "realty_subject")
    titles = [title.text.replace("제목 :", "").strip() for title in title_elements]

    # 장점
    advantages = [advantage.text.replace("장점 : ", "").strip() for advantage in driver.find_elements(By.ID, "base_keyword")]

    for i in range(len(types)):
        csv_writer.writerow([
            types[i] if i < len(types) else '',
            address1[i] if i < len(address1) else '',
            address2[i] if i < len(address2) else '',
            address3[i] if i < len(address3) else '',
            sizes[i] if i < len(sizes) else '',
            prices[i] if i < len(prices) else '',
            rooms[i] if i < len(rooms) else '',
            roads[i] if i < len(roads) else '',
            clean_parks[i] if i < len(clean_parks) else '',
            counts[i] if i < len(counts) else '',
            titles[i] if i < len(titles) else '',
            advantages[i] if i < len(advantages) else '',
            urls[i] if i < len(urls) else '',
        ])


In [96]:
i = 1

with open('RealEaste119.csv', 'w', newline='', encoding='cp949') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(['매물종류', '소재지(1)', '소재지(2)', '소재지(3)', '면적(연, 토지)', '가격', '층수(방 수)', '진입, 인접도로', '이삿날(주차)', '조회수', '제목', '장점', 'url'])

    while True:
        crawl_page(driver, csv_writer)

        try:
            # 다음 페이지로 이동
            next_button = driver.find_elements(By.CLASS_NAME, 'paging_item')[1]
            print(f'{i} 페이지 완료')
            i += 1
            next_button.click()
            time.sleep(2) #페이지 로드 대기 시간
        except Exception as e:
            print("더 이상 페이지가 없습니다.")
            break

print('크롤링 완료')

1 페이지 완료
2 페이지 완료
3 페이지 완료
4 페이지 완료
5 페이지 완료
6 페이지 완료
7 페이지 완료
8 페이지 완료
9 페이지 완료
10 페이지 완료
11 페이지 완료
12 페이지 완료
13 페이지 완료
14 페이지 완료
15 페이지 완료
16 페이지 완료
17 페이지 완료
18 페이지 완료
19 페이지 완료
20 페이지 완료
21 페이지 완료
22 페이지 완료
23 페이지 완료
24 페이지 완료
25 페이지 완료
26 페이지 완료
27 페이지 완료
28 페이지 완료
29 페이지 완료
30 페이지 완료
31 페이지 완료
32 페이지 완료
33 페이지 완료
34 페이지 완료
더 이상 페이지가 없습니다.
크롤링 완료
